# Variational Quantum Regression

$$
\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}
\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}
$$

## Introduction

Here we create a protocol for linear regression which can exploit the properties of a quantum computer. For this problem, we assume that we have two data sets, x and y, where x is the independent data and y is the dependent data. There are N data points in each data set. We first want to fit this data to the following equation:

$$y = ax + b$$

and then we will include higher powers of x. First, we will theoretically explore this proposed algorithm, and then we will tweak the code slightly so that it can be run on a real quantum computer. This algorithm has no known advantage over the most widely-used classical algorithm ([Least Squares Method](https://doi.org/10.1016/j.proeng.2012.09.545)), but does nicely demonstrate the different elements of variational quantum algorithms.

## Variational Quantum Computing

Variational quantum computing exploits the advantages of both classical computing and quantum computing. In a very general sense, we propose an initial solution to a problem, called an ansatz. In our case our ansatz will be an ansatz parametrised by a and b. We then prepare our qubits (the quantum equivalent of bits on a normal computer) and test how good the ansatz is, using the quantum computer. Testing the ansatz equates to minimising a cost function. We feed the result of this cost function back to the classical computer, and use some classical optimisers to improve on our ansatz, i.e. our initial guesses for a and b. We repeat this process until the ansatz is good enough within some tolerance. 
![title](images/vlinreg_circuit.png)

## Translate to Quantum Domain

We now need to explore how we will translate the data set, y, onto a quantum computer. Let us think of y as a length N vector. The easiest way to encode this data set onto a quantum computer is by initialising qubits in the state $\ket{y}$, where 

$$\ket{y} = \frac{1}{C_y}\vec{y}$$

and $C_y$ is a normalisation factor.

Now we propose a trial solution, or ansatz, which is parametrised by a and b, as follows:

$$\ket{\Phi} = \frac{1}{C_{\Phi}}(a\vec{x} + b)$$ 

where $C_{\Phi}$ is again a normalisation factor.

Due to the definition of the tensor product and the fact that the general statevector of a single qubit is a vector of length 2, $n$ qubits can encode length-$2^n$ vectors. 

### Cost Function

Our proposed cost function, which we wish to minimise is equal to

$$C_P = \big(1 - \braket{y}{\Phi}\big)^2$$

This computes the normalised fidelity (similarity) of $\ket{y}$ and $\ket{\Phi}$. We see that if $\ket{y}$ and $\ket{\Phi}$ are equal, our cost function will equal 0, otherwise it will be greater than 0. Thus, we need to compute this cost function with our quantum hardware, and couple it with classical minimising algorithms. 

### Computing Inner Products on a Quantum Computer

It is clear we now need a quantum algorithm for computing inner products. Let us go through the theory of computing the inner product $\braket{x}{y}$ here, which will be translated to quantum hardware in a couple of sections. 

Firstly, assume we have a state:

$$ \ket{\phi} = \frac{1}{\sqrt{2}}\big(\ket{0}\ket{x} + \ket{1}\ket{y}\big) $$

where we want to find the inner product, $\braket{x}{y}$. Applying a Hadamard gate on the first qubit, we find:

$$ \ket{\tilde{\phi}} = \frac{1}{2}\Big(\ket{0}\big(\ket{x}+\ket{y}\big) + \ket{1}\big(\ket{x}-\ket{y}\big)\Big) $$

This means that the probability to measure the first qubit as $\ket{0}$ in the computational basis equals:

$$ P(0) = \frac{1}{2}\Big(1+Re\big[\braket{x}{y}\big]\Big) $$

This follows because:

$$
\begin{aligned}
    P(0) &= \Big|\bra{0}\otimes\mathbb{1}\ket{\tilde{\phi}}\Big|^2 \\
    &= \frac{1}{4}\Big|\ket{x}+\ket{y}\Big|^2 \\
    &= \frac{1}{4}\big(\braket{x}{x}+\braket{x}{y}+\braket{y}{x}+\braket{y}{y}\big) \\
    &= \frac{1}{4}\Big(2 + 2 Re\big[\braket{x}{y}\big]\Big) \\
    &= \frac{1}{2}\Big(1+Re\big[\braket{x}{y}\big]\Big)
\end{aligned}
$$

After a simple rearrangement, we see that

$$Re\big[\braket{x}{y}\big] = 2P(0) - 1$$

It follows from a similar logic that if we apply a phase rotation on our initial state:

$$ \ket{\phi} = \frac{1}{\sqrt{2}}\big(\ket{0}\ket{x} -i \ket{1}\ket{y}\big) $$

then the probability of the same measurement:

$$ P(0) = \frac{1}{2}\Big(1+Im\big[\braket{x}{y}\big]\Big) $$

We can then combine both probabilities to find the true $\braket{x}{y}$. For this work, we assume that our states are fully real, and so just need the first measurement.

## Code Implementation - Theoretical Approach

It should be noted here that qiskit orders its qubits with the last qubit corresponding to the left of the tensor product. For this run through, we are computing the inner product of length-8 vectors. Thus, we require 4 qubits ($8 + 8 = 16 = 2^4$) to encode the state:

$$
\begin{aligned}
    \ket{\phi} &= \frac{1}{\sqrt{2}}(\ket{0}\ket{x} + \ket{1}\ket{y}) \\ &= \frac{1}{\sqrt{2}}\left(\begin{bmatrix}1\\0\end{bmatrix}\otimes\begin{bmatrix}x_1\\x_2\\\vdots\\x_n \end{bmatrix} +\begin{bmatrix}0\\1\end{bmatrix}\otimes\begin{bmatrix}y_1\\y_2\\\vdots\\y_n \end{bmatrix} \right) \\
    &= \frac{1}{\sqrt{2}}\left(\begin{bmatrix}x_1\\x_2\\\vdots\\x_n \\y_1\\y_2\\\vdots\\y_n \end{bmatrix} \right)
\end{aligned}
$$

Finally, in order to measure the probability of measuring the bottom (leftmost) qubit as $\ket{0}$ in the computational basis, we can find the exact theoretical value by finding the resultant statevector and summing up the amplitude squared of the first $2^{n-1}$ entries (i.e. half of them). On a real quantum computer, we would just have to perform the actual measurement many times over, and compute the probability that way. We will show the theoretical approach in practice first. 

In [ ]:
#conda activate quantum (conda env)
# importing necessary packages
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import BasicAer, execute #Aer, execute
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import torch
from transformers import AutoTokenizer, AutoModel
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.utils import QuantumInstance



/var/folders/h5/0m1hlmsj0g1_y_yfsfmvlqm80000gn/T/ipykernel_53903/1193852245.py:5: DeprecationWarning: BasicAer is deprecated since Qiskit 0.46 and will be removed in Qiskit 1.0. The BasicAer (qiskit.providers.basicaer) module has been superseded by  qiskit.providers.basic_provider, and all its classes have been renamed to follow a new naming convention. More information and migration guidelines can be found in the 0.46 API docs for BasicAer.
  from qiskit import BasicAer, execute #Aer, execute


In [2]:
from transformers import AutoTokenizer, AutoModel
import torch

# Test sentences
test_sentences = [
    "I have two cats, a black one named Tom and a white one named Jerry.",
    "Tom likes to chase Jerry around the house.",
    "The white house is a big house.",
    "NYU is a university in New York City.",
    "Jerry is a professor at NYU.",
    "Tom is twelve years old.",
    "Jerry is a cat."
]

class SentenceTransformer:
    def __init__(self, model_name, **kwargs):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, **kwargs)
        self.model = AutoModel.from_pretrained(model_name, **kwargs)
        
    def encode(self, sentences):
        encoded_input = self.tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        return model_output.last_hidden_state.mean(dim=1)

    def cosine_similarity(self, sent1, sent2):
        emb1, emb2 = self.encode([sent1, sent2])
        return torch.nn.functional.cosine_similarity(emb1, emb2, dim=0).item()

class TestSentenceTransformers:
    def __init__(self):
        self.models = {
            'DistilBERT': SentenceTransformer('distilbert-base-uncased'),
            'BERT': SentenceTransformer('bert-base-uncased'),
            'RoBERTa': SentenceTransformer('roberta-base'),
            'XLM-RoBERTa': SentenceTransformer('xlm-roberta-base'),
            'Jina v3': SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)
        }

    def test_encoding(self):
        for name, model in self.models.items():
            print(f"Testing encoding for {name}:")
            embeddings = model.encode(test_sentences)
            print(f"Shape of embeddings: {embeddings.shape}")
            print(f"Sample embedding (first 5 dimensions): {embeddings[0][:5]}\n")
        return embeddings

    def test_similarity(self):
        for name, model in self.models.items():
            print(f"Testing similarity for {name}:")
            sim = model.cosine_similarity(test_sentences[0], test_sentences[1])
            print(f"Similarity between '{test_sentences[0]}' and '{test_sentences[1]}': {sim:.4f}\n")

# Run tests
tester = TestSentenceTransformers()
print("Testing encoding:")
wvs = tester.test_encoding()
print("\nTesting similarity:")
tester.test_similarity()

/Users/zongruipei/anaconda3/envs/quantum/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Testing encoding:
Testing encoding for DistilBERT:
Shape of embeddings: torch.Size([7, 768])
Sample embedding (first 5 dimensions): tensor([-0.1944,  0.0392, -0.1053, -0.1830,  0.5706])

Testing encoding for BERT:
Shape of embeddings: torch.Size([7, 768])
Sample embedding (first 5 dimensions): tensor([-0.1915,  0.0796, -0.1610, -0.3328,  0.6095])

Testing encoding for RoBERTa:
Shape of embeddings: torch.Size([7, 768])
Sample embedding (first 5 dimensions): tensor([-0.1294,  0.1826,  0.0787, -0.1806,  0.1023])

Testing encoding for XLM-RoBERTa:
Shape of embeddings: torch.Size([7, 768])
Sample embedding (first 5 dimensions): tensor([-0.0292,  0.0627, -0.0264,  0.0488,  0.0653])

Testing encoding for Jina v3:
Shape of embeddings: torch.Size([7, 1024])
Sample embedding (first 5 dimensions): tensor([-0.1125, -2.8233,  0.9402,  2.3514,  1.0594])


Testing similarity:
Testing similarity for DistilBERT:
Similarity between 'I have two cats, a black one named Tom and a white one named Jerry.' an

Now, let's draw the required diagram for theoretically computing the inner product of any two states. Note that the only difference between this circuit diagram and the real, practical diagram for actually running on a quantum computer is that we do not measure the left-most qubit in the computational basis. Again, note that the left-most qubit corresponds to the bottom qubit.

In [3]:
print(wvs)

tensor([[-0.1125, -2.8233,  0.9402,  ..., -0.1039, -0.2443,  0.0464],
        [ 0.9059, -1.6185,  2.1889,  ..., -0.0876, -0.8054,  0.1452],
        [ 0.2599, -1.7680,  0.7608,  ..., -0.1531, -0.7527,  0.1791],
        ...,
        [ 1.8343, -2.0368,  1.8156,  ..., -0.0139, -0.3792,  0.4629],
        [ 1.0413, -2.1509,  2.2642,  ..., -0.2211, -0.6279,  0.4043],
        [ 0.9942, -1.9542,  1.6484,  ..., -0.0633, -0.5077,  0.1152]])


In [7]:
x,y=np.array(wvs[0]),np.array(wvs[1])
print(x,y)
N = len(x)              
nqubits = math.ceil(np.log2(N))    # compute how many qubits needed to encode either x or y

xnorm = np.linalg.norm(x)          # normalise vectors x and y
ynorm = np.linalg.norm(y)
x = x/xnorm
y = y/ynorm

circ = QuantumCircuit(nqubits+1)   # create circuit
vec = np.concatenate((x,y))/np.sqrt(2)    # concatenate x and y as above, with renormalisation

vec = np.concatenate((x, y)).astype(np.float64)
vec = vec / np.linalg.norm(vec)   # ensure exact normalization

circ.initialize(vec, range(nqubits + 1))

circ.initialize(vec, range(nqubits+1))
circ.h(nqubits)                    # apply hadamard to bottom qubit

circ.draw()                        # draw the circuit

[-0.11248491 -2.8232753   0.9402168  ... -0.10385526 -0.2442747
  0.04638601] [ 0.9058683  -1.6184831   2.1889396  ... -0.0876447  -0.80536515
  0.14518017]


»
 q_0: »
      »
 q_1: »
      »
 q_2: »
      »
 q_3: »
      »
 q_4: »
      »
 q_5: »
      »
 q_6: »
      »
 q_7: »
      »
 q_8: »
      »
 q_9: »
      »
q_10: »
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [10]:
#Creates a quantum circuit to calculate the inner product between two normalised vectors

def inner_prod(vec1, vec2):
    #first check lengths are equal
    if len(vec1) != len(vec2):
        raise ValueError('Lengths of states are not equal')
        
    circ = QuantumCircuit(nqubits+1)
    vec = np.concatenate((vec1,vec2))/np.sqrt(2)
    vec = np.concatenate((x, y)).astype(np.float64)
    vec = vec / np.linalg.norm(vec)   # ensure exact normalization

    circ.initialize(vec, range(nqubits+1))
    circ.h(nqubits)

    #backend = Aer.get_backend('statevector_simulator')
    backend = BasicAer.get_backend('statevector_simulator') #get_backend('qasm_simulator')
    job = execute(circ, backend, backend_options = {"zero_threshold": 1e-20})

    result = job.result()
    o = np.real(result.get_statevector(circ))

    m_sum = 0
    for l in range(N):
        m_sum += o[l]**2
        
    return 2*m_sum-1

x,y=np.array(wvs[0]),np.array(wvs[1])
print(x,y)
N = len(x)  

nqubits = math.ceil(np.log2(N))
xnorm = np.linalg.norm(x)
ynorm = np.linalg.norm(y)
x = x/xnorm
y = y/ynorm

print("x: ", x)
print()
print("y: ", y)
print()
print("The inner product of x and y equals: ", inner_prod(x,y))

[-0.11248491 -2.8232753   0.9402168  ... -0.10385526 -0.2442747
  0.04638601] [ 0.9058683  -1.6184831   2.1889396  ... -0.0876447  -0.80536515
  0.14518017]
x:  [-0.0055803  -0.14006075  0.04664351 ... -0.00515219 -0.0121183
  0.00230118]

y:  [ 0.04505726 -0.08050223  0.10887633 ... -0.00435939 -0.0400583
  0.00722116]



/var/folders/h5/0m1hlmsj0g1_y_yfsfmvlqm80000gn/T/ipykernel_53903/2590710544.py:18: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(circ, backend, backend_options = {"zero_threshold": 1e-20})


The inner product of x and y equals:  0.6504027984628389


In [ ]:
## Quantum Sentence Similarity (Extension)


In [ ]:
# ------------------------------------------------------------------
# 1. Load a sentence encoder (Transformers)
# ------------------------------------------------------------------

# Sentence-transformer / BERT-style model
SENTENCE_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

tokenizer = AutoTokenizer.from_pretrained(SENTENCE_MODEL_NAME)
sentence_model = AutoModel.from_pretrained(SENTENCE_MODEL_NAME)

def get_sentence_embedding(sentence: str) -> np.ndarray:
    """
    Convert a sentence into a single embedding vector using mean pooling
    over token embeddings from the Transformer model.
    """
    inputs = tokenizer(
        sentence,
        return_tensors="pt",
        truncation=True,
        padding=True
    )

    with torch.no_grad():
        outputs = sentence_model(**inputs)

    # Token embeddings: [1, seq_len, hidden_dim]
    token_embeddings = outputs.last_hidden_state
    attention_mask = inputs["attention_mask"]

    # Mask padding tokens
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

    # Mean pooling
    summed = (token_embeddings * mask).sum(dim=1)
    counts = mask.sum(dim=1)
    sentence_embedding = summed / counts

    return sentence_embedding[0].cpu().numpy()


# ------------------------------------------------------------------
# 2. Map embeddings to quantum features
# ------------------------------------------------------------------

NUM_QUBITS = 4

def embedding_to_features(vec: np.ndarray, num_features: int = NUM_QUBITS) -> np.ndarray:
    """
    Reduce a high-dimensional embedding to a fixed number of features
    and scale values to [0, pi] for use in a quantum feature map.
    """
    # Truncate or pad
    if vec.shape[0] > num_features:
        v = vec[:num_features]
    else:
        v = np.pad(vec, (0, num_features - vec.shape[0]), mode="constant")

    # Min-max scale to [0, pi]
    v = v - v.min()
    vmax = v.max()
    if vmax > 0:
        v = v / vmax * np.pi

    return v.astype(np.float64)


# ------------------------------------------------------------------
# 3. Set up QuantumKernel with ZZFeatureMap
# ------------------------------------------------------------------

backend = BasicAer.get_backend("qasm_simulator")

feature_map = ZZFeatureMap(
    feature_dimension=NUM_QUBITS,
    reps=2,
    entanglement="linear"
)

quantum_instance = QuantumInstance(
    backend=backend,
    shots=2048
)

quantum_kernel = QuantumKernel(
    feature_map=feature_map,
    quantum_instance=quantum_instance
)


# ------------------------------------------------------------------
# 4. Quantum sentence similarity
# ------------------------------------------------------------------

def quantum_sentence_similarity(sent_a: str, sent_b: str):
    """
    Compute a quantum kernel-based similarity score between two sentences.
    """
    emb_a = get_sentence_embedding(sent_a)
    emb_b = get_sentence_embedding(sent_b)

    feat_a = embedding_to_features(emb_a)
    feat_b = embedding_to_features(emb_b)

    X = np.vstack([feat_a, feat_b])

    K = quantum_kernel.evaluate(X, X)

    k00 = np.real(K[0, 0])
    k11 = np.real(K[1, 1])
    k01 = np.real(K[0, 1])

    similarity = k01 / np.sqrt(k00 * k11)

    return float(similarity), K


def quantum_sentence_similarity_matrix(sentences):
    """
    Compute an N x N quantum similarity matrix for a list of sentences.
    """
    embeddings = [get_sentence_embedding(s) for s in sentences]
    features = np.vstack([embedding_to_features(e) for e in embeddings])

    K = quantum_kernel.evaluate(features, features)

    diag = np.real(np.diag(K))
    norm = np.sqrt(diag)
    norm[norm == 0] = 1.0

    S = np.real(K) / (norm[:, None] * norm[None, :])

    return S


In [ ]:
s1 = "Quantum computing improves machine learning models."
s2 = "Quantum algorithms can enhance AI systems."

sim, K = quantum_sentence_similarity(s1, s2)
print("Quantum similarity score:", sim)


## Acknowledgements

I would like to thank Dr. Lee O'Riordan for his supervision and guidance on this work. The work was mainly inspired by work presented in the research paper "Variational Quantum Linear Solver: A Hybrid Algorithm for Linear Systems", written by Carlos Bravo-Prieto, Ryan LaRose, M. Cerezo, Yiğit Subaşı, Lukasz Cincio, and Patrick J. Coles, which is available at this [link](https://arxiv.org/abs/1909.05820). I would also like to thank the Irish Centre for High End Computing for allowing me to access the national HPC infrastructure, Kay.